In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
# from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=25)

PATH='data/favorita/'

In [3]:
!ls {PATH}

holidays_events.csv        stores.csv
items.csv                  test.csv
oil.csv                    train.csv
rf_rnn                     train_six_months_full_data
sample_submission.csv      transactions.csv


In [4]:
from IPython.display import HTML

# Separate into 2 models

1. Store/Item/Date combinations exist in test -> DOW/DOM averages
2. Store/Item/Date combinations do not exist -> Remove store_nbr, item_nbr, use family,cluster,etc. to generalize  
    [0.7118, 0.7376, 0.3322, 0.28510000000000002]

https://www.kaggle.com/aharless/dissecting-ceshine-lee-s-lgbm-kernel

https://www.kaggle.com/captcalculator/a-very-extensive-favorita-exploratory-analysis/notebook

## Find Validation Set which best mimics % of missing Test combos 

## Moving averages
https://www.kaggle.com/paulorzp/log-ma-and-days-of-week-means-lb-0-529/code

In [ ]:
dtypes = {'id':'uint32', 'item_nbr':'uint32', 'store_nbr':'uint8', 'onpromotion':'bool'}

In [ ]:
train = pd.read_csv(f'{PATH}train.csv', usecols=[1, 2, 3, 4, 5], dtype=dtypes,
            converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
            parse_dates=['date'], skiprows=range(1, 114176251))  # header counts as row 0

# 86672217 => Skip dates before 2016-08-01
# 114176250 => 2017-05-01

In [ ]:
#Load test
test = pd.read_csv(f'{PATH}test.csv', dtype=dtypes, parse_dates=['date'])

# Join tables and remove item_nbr and store_nbr -> generalize

In [ ]:
items = pd.read_csv(f'{PATH}items.csv', low_memory=False)
stores = pd.read_csv(f'{PATH}stores.csv', low_memory=False)
# holidays = pd.read_csv(f'{PATH}holidays_events.csv', parse_dates=['date'], low_memory=False)

### Stores

In [ ]:
stores.head()

In [ ]:
train = pd.merge(train, stores, how='left', on=['store_nbr'])
test = pd.merge(test, stores, how='left', on=['store_nbr'])

In [ ]:
del stores

### Items

In [ ]:
items.head()

In [ ]:
train = pd.merge(train, items, how='left', on=['item_nbr'])
test = pd.merge(test, items, how='left', on=['item_nbr'])

In [ ]:
del items

### Remove 'store_nbr' and 'item_nbr'

In [ ]:
train.drop(['store_nbr','item_nbr'], axis=1, inplace=True)
test.drop(['store_nbr','item_nbr'], axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

### Separate validation set before filling in missing data

**Problem:**  
missing ~25% of the ['item_nbr', 'store_nbr'] combinations of training set  
Need to create a validation set with similar circumstances...

In [ ]:
# last 2 weeks
valid = train[train.date>='2017-07-16']

In [ ]:
# drop valid from train
train = train[:-(len(valid))]; train.tail()

In [ ]:
valid = valid.reset_index(drop=True)

valid['dow'] = valid['date'].dt.dayofweek
valid['dom'] = valid['date'].dt.day

### create missing rows (items,stores,dates)

In [ ]:
# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
).reset_index()

del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
train.loc[:, 'onpromotion'].fillna(False, inplace=True) # fill NaNs
lastdate = train.iloc[train.shape[0]-1].date

In [ ]:
train['dow'] = train['date'].dt.dayofweek
train['dom'] = train['date'].dt.day

In [ ]:
os.makedirs(f'{PATH}rf_rnn', exist_ok=True)
train.to_feather(f'{PATH}rf_rnn/train_3_mo')

In [ ]:
# train = pd.read_feather(f'{PATH}rf_rnn/train_full_year_incomplete')
# dtypes = {'id':'uint32', 'item_nbr':'uint32', 'store_nbr':'uint8', 'onpromotion': 'bool'}
lastdate = train.iloc[train.shape[0]-1].date

In [ ]:
#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(
        ['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw').reset_index()
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(
        ['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk').reset_index()

In [ ]:
#Days of Month Means
ma_dm = train[['item_nbr','store_nbr','dom','unit_sales']].groupby(
    ['item_nbr','store_nbr','dom'])['unit_sales'].mean().to_frame('madm').reset_index()
ma_mo = ma_dm[['item_nbr','store_nbr','madm']].groupby(
    ['item_nbr','store_nbr'])['madm'].mean().to_frame('mamo').reset_index()

In [ ]:
from datetime import timedelta

#Moving Averages
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(
        ['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais')

for i in [55,34,21,13,8,5,3,2,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

del tmp,tmpg

ma_is['mais']=ma_is.median(axis=1)
ma_is.reset_index(inplace=True)
ma_is.drop(list(ma_is.columns.values)[3:],1,inplace=True)

In [ ]:
#Load test
test = pd.read_csv(f'{PATH}test.csv', dtype=dtypes, parse_dates=['date'])
# test['dow'] = test['date'].dt.dayofweek
# test['dom'] = test['date'].dt.day

In [ ]:
# merge moving averages onto test df
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_mo, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])
test = pd.merge(test, ma_dm, how='left', on=['item_nbr','store_nbr','dom'])

# merge moving averages onto valid df
valid = pd.merge(valid, ma_is, how='left', on=['item_nbr','store_nbr'])
valid = pd.merge(valid, ma_wk, how='left', on=['item_nbr','store_nbr'])
valid = pd.merge(valid, ma_mo, how='left', on=['item_nbr','store_nbr'])
valid = pd.merge(valid, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])
valid = pd.merge(valid, ma_dm, how='left', on=['item_nbr','store_nbr','dom'])

# merge moving averages onto train df
train = pd.merge(train, ma_is, how='left', on=['item_nbr','store_nbr'])
train = pd.merge(train, ma_wk, how='left', on=['item_nbr','store_nbr'])
train = pd.merge(train, ma_mo, how='left', on=['item_nbr','store_nbr'])
train = pd.merge(train, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])
train = pd.merge(train, ma_dm, how='left', on=['item_nbr','store_nbr','dom'])

In [ ]:
DataFrameSummary(test).summary()

In [ ]:
val = valid[(valid['date']>='2017-7-26') & (valid['date']<='2017-8-9')]

In [ ]:
DataFrameSummary(val).summary()

In [ ]:
train.to_feather(f'{PATH}rf_rnn/train_dom_dow_w_avgs')
valid.to_feather(f'{PATH}rf_rnn/valid_dom_dow_w_avgs')
test.to_feather(f'{PATH}rf_rnn/test_dom_dow_w_avgs')

## Calculate averages for train/test

In [ ]:
# avg price * dow multiplier
train['avg_dow'] = train.mais
pos_idx = train['mawk'] > 0  # avoid division by zero error
train_pos = train.loc[pos_idx]
train.loc[pos_idx, 'avg_dow'] = train_pos['mais'] * train_pos['madw'] / train_pos['mawk']
# train.loc[:, 'avg_dow'].fillna(train['avg_dow'].median(), inplace=True)  # fill w/ median instead of 0
train.drop(['mawk', 'madw'], axis=1, inplace=True)

In [ ]:
# avg price * dom multiplier
train['avg_dom'] = train.mais
pos_idx = train['mamo'] > 0  # avoid division by zero error
train_pos = train.loc[pos_idx]
train.loc[pos_idx, 'avg_dom'] = train_pos['mais'] * train_pos['madm'] / train_pos['mamo']
# train.loc[:, 'avg_dom'].fillna(train['avg_dom'].median(), inplace=True)
train.drop(['mais', 'mamo', 'madm'], axis=1, inplace=True); train.head()

In [ ]:
test['avg_dow'] = test.mais
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'avg_dow'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
# test.loc[:, 'avg_dow'].fillna(test['avg_dow'].median(), inplace=True)
test.drop(['mawk', 'madw'], axis=1, inplace=True)

In [ ]:
test['avg_dom'] = test.mais
pos_idx = test['mamo'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'avg_dom'] = test_pos['mais'] * test_pos['madm'] / test_pos['mamo']
# test.loc[:, "avg_dom"].fillna(test['avg_dom'].median(), inplace=True)
test.drop(['mais', 'mamo', 'madm'], axis=1, inplace=True); test.head()

In [ ]:
valid['avg_dow'] = valid.mais
pos_idx = valid['mawk'] > 0
valid_pos = valid.loc[pos_idx]
valid.loc[pos_idx, 'avg_dow'] = valid_pos['mais'] * valid_pos['madw'] / valid_pos['mawk']
# valid.loc[:, 'avg_dow'].fillna(valid['avg_dow'].median(), inplace=True)
valid.drop(['mawk', 'madw'], axis=1, inplace=True)

In [ ]:
valid['avg_dom'] = valid.mais
pos_idx = valid['mamo'] > 0
valid_pos = valid.loc[pos_idx]
valid.loc[pos_idx, 'avg_dom'] = valid_pos['mais'] * valid_pos['madm'] / valid_pos['mamo']
# valid.loc[:, "avg_dom"].fillna(valid['avg_dom'].median(), inplace=True)
valid.drop(['mais', 'mamo', 'madm'], axis=1, inplace=True); valid.head()

In [ ]:
del pos_idx, train_pos, test_pos

In [ ]:
train.to_feather(f'{PATH}rf_rnn/train_w_averages_incomplete')
test.to_feather(f'{PATH}rf_rnn/test_w_averages_incomplete')

## Establish baseline (moving averages)

In [ ]:
# Root Mean Squared Logarathimic Error
# kaggle definition
# np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean() ** 0.5

def rmsle(pred, targ):
    # pred and targs are both log1p values already
    mean_diff = np.square(pred - targ).mean()
    return round(math.sqrt(mean_diff), 4)

In [ ]:
# testing 25% more for promotion items
#dow
train['avg_dow_promo'] = train['avg_dow']
train.loc[train['onpromotion'] == True, 'avg_dow_promo'] *= 1.25

valid['avg_dow_promo'] = valid['avg_dow']
valid.loc[valid['onpromotion'] == True, 'avg_dow_promo'] *= 1.25
# dom
train['avg_dom_promo'] = train['avg_dom']
train.loc[train['onpromotion'] == True, 'avg_dom_promo'] *= 1.25

valid['avg_dom_promo'] = valid['avg_dom']
valid.loc[valid['onpromotion'] == True, 'avg_dom_promo'] *= 1.25

# drop avg_dow_promo columns from train/valid
train.drop('avg_dow_promo', axis=1, inplace=True)
valid.drop('avg_dow_promo', axis=1, inplace=True)
train.drop('avg_dom_promo', axis=1, inplace=True)
valid.drop('avg_dom_promo', axis=1, inplace=True)

In [ ]:
# The baseline predictions on full training set
print(rmsle(train.avg_dow, train.unit_sales))  #=> 0.7158
print(rmsle(train.avg_dom, train.unit_sales))  #=> 0.337
# 0.5269
# 0.5264

# baseline w/ promo multiplier
print(rmsle(train.avg_dow_promo, train.unit_sales))  #=> 0.7133
print(rmsle(train.avg_dom_promo, train.unit_sales))
# 0.5292
# 0.5238

# 0.5985
# 0.5699
# 0.5993
# 0.5741

In [ ]:
# baseline predictions on validation set
print(rmsle(valid.avg_dow, valid.unit_sales))  #=> 0.5138
print(rmsle(valid.avg_dom, valid.unit_sales))  #=> 0.2611
# 0.4915
# 0.483

# baseline w/ promo multiplier
print(rmsle(valid.avg_dow_promo, valid.unit_sales))  #=> 0.5116
print(rmsle(valid.avg_dom_promo, valid.unit_sales))
# 0.5013
# 0.4891

# 0.663
# 0.7611
# 0.6654
# 0.762

In [ ]:
# average dom/dow for full and valid sets
print(rmsle(train[['avg_dow', 'avg_dom']].mean(axis=1), train.unit_sales))
print(rmsle(valid[['avg_dow', 'avg_dom']].mean(axis=1), valid.unit_sales))

It appears that avg dow/dom is the best baseline predictor...

In [ ]:
DataFrameSummary(valid).summary()

In [ ]:
DataFrameSummary(test).summary()

### Submit test on kaggle

In [ ]:
# set unit_sales
test['unit_sales'] = test[['avg_dom','avg_dow']].mean(axis=1)
# test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.25

# need to convert unit_sales back from log1p
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values

In [ ]:
SUBM = f'{PATH}rf_rnn/subm/'
os.makedirs(SUBM, exist_ok=True)

test.to_csv(f'{SUBM}avg_dow_dom_v2.csv.gz', columns=['id','unit_sales'], index=False, compression='gzip')

In [ ]:
!kaggle competitions submit -c favorita-grocery-sales-forecasting -f {SUBM}avg_dow_dom_v2.csv.gz -m "dow and dom averages v2"
#=> 0.537  avg_dow w/ promo multiplier
#=> 0.718  dow/dom averages -- why you so bad!?
#=> 0.718  dow/dom_v2 averages

## Feature Engineering

In [5]:
train = pd.read_feather(f'{PATH}rf_rnn/train_w_averages')
test = pd.read_feather(f'{PATH}rf_rnn/test_w_averages')

In [ ]:
# NOTE: need to convert dates back to datetime?!
train['date'] = train['date'].astype('datetime64[ns]')
test['date'] = test['date'].astype('datetime64[ns]')

In [ ]:
items = pd.read_csv(f'{PATH}items.csv', low_memory=False)
stores = pd.read_csv(f'{PATH}stores.csv', low_memory=False)
# holidays = pd.read_csv(f'{PATH}holidays_events.csv', parse_dates=['date'], low_memory=False)

### Stores

In [ ]:
stores.head()

In [ ]:
train = pd.merge(train, stores, how='left', on=['store_nbr'])
test = pd.merge(test, stores, how='left', on=['store_nbr'])

In [ ]:
del stores

### Holidays

In [ ]:
# looks like we only need to worry about local holidays...
holidays[(holidays['date'] >= '2017-08-16') & (holidays['date'] <= '2017-08-31')]

In [ ]:
local_holidays = holidays.loc[(holidays['locale'] == 'Local') & (holidays['transferred'] == False)].copy()
local_holidays['holiday'] = True
local_holidays = local_holidays.rename(index=str, columns={"locale_name": "city"}).drop(
    ['type','locale','description','transferred'], axis=1)
local_holidays.head()

In [ ]:
train = pd.merge(train, local_holidays, how='left', on=['date','city'])
test = pd.merge(test, local_holidays, how='left', on=['date','city'])

In [ ]:
train.loc[:, 'holiday'].fillna(False, inplace=True) # fill NaNs
test.loc[:, 'holiday'].fillna(False, inplace=True)

In [ ]:
del holidays, local_holidays

### Items

In [ ]:
items.head()

In [ ]:
train = pd.merge(train, items, how='left', on=['item_nbr'])
test = pd.merge(test, items, how='left', on=['item_nbr'])

In [ ]:
pd.isnull(train).any()

In [ ]:
del items

In [ ]:
train.to_feather(f'{PATH}rf_rnn/train_w_features')
test.to_feather(f'{PATH}rf_rnn/test_w_features')

## Data prep - handle categorical variables

In [ ]:
# train = pd.read_feather(f'{PATH}rf_rnn/train_w_features')
test = pd.read_feather(f'{PATH}rf_rnn/test_w_features')

In [ ]:
# add date info
add_datepart(train, 'date')
add_datepart(test, 'date')

In [ ]:
test.columns.values

In [ ]:
test.drop(np.append(test.columns.values[15:], ['Year', 'Month', 'Week']), axis=1, inplace=True)
train.drop(np.append(test.columns.values[15:], ['Year', 'Month', 'Week']), axis=1, inplace=True)

In [ ]:
cat_vars = ['onpromotion', 'city', 'state', 'type', 'cluster','family', 'class', 'perishable']

In [ ]:
for v in cat_vars:
    train[v] = train[v].astype('category').cat.as_ordered()

In [ ]:
apply_cats(test, train)

In [ ]:
train.to_feather(f'{PATH}rf_rnn/train_cats')
test.to_feather(f'{PATH}rf_rnn/test_cats')

In [ ]:
train = pd.read_feather(f'{PATH}rf_rnn/train_cats')
test = pd.read_feather(f'{PATH}rf_rnn/test_cats')

In [ ]:
train.drop(train.columns.values[11:],1,inplace=True)
train.head()

In [ ]:
# test.drop('Dayofyear',1,inplace=True)
test.head()

## Separate target from df

In [ ]:
# last 3.5 months
train = pd.read_feather(f'{PATH}rf_rnn/train_w_categories')
# test = pd.read_feather(f'{PATH}rf_rnn/test_w_categories')

In [ ]:
df, y, nas, mapper = proc_df(train, 'unit_sales', do_scale=True)

## Split training/validation

In [ ]:
# Using Skicit-learn to split data into training and testing sets
# from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
# x_trn, x_val, y_trn, y_val = train_test_split(df, y, test_size = 0.20, random_state = 42)

# good validation set => 2017-7-26 -> 2017-8-9
# IDs => [9119925:10700781] (3.5 month dataset)
# IDs => [18743184:22012344] (18743184 - 22012343)
# len(test) => 3370464

def split_val(df,a,b):
    val = df[a:b].copy()
    trn = df.drop(df.index[a:b]).copy()
    return trn, val

# x_trn,x_val = split_val(df,18743184,22012344)

def split_val_arr(arr,a,b):
    val = arr[a:b].copy()
    trn = np.delete(arr, slice(a,b))
    return trn, val

# y_trn,y_val = split_val_arr(y,18743184,22012344)

In [ ]:
x_trn,x_val = split_val(df,9119925,10700781)
y_trn,y_val = split_val_arr(y,9119925,10700781)

In [ ]:
x_trn.shape, x_val.shape, y_trn.shape, y_val.shape

In [ ]:
del train, df, y

## Train model

In [ ]:
def rmsle(pred, targ):
    # pred and targs are both log1p values already
    mean_diff = np.square(pred - targ).mean()
    return round(math.sqrt(mean_diff), 4)

def print_score(m):
    res = [rmsle(m.predict(x_trn), y_trn), rmsle(m.predict(x_val), y_val),
                round(m.score(x_trn, y_trn),4), round(m.score(x_val, y_val),4)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
    
# m.score => coefficient of determination (R^2)
#   proportion of the variance in the dependent variable that is predictable from the independent variable(s)
#   ratio of how much better the model is than the mean prediction (0);  1: perfect;  -*: worse than the mean

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
set_rf_samples(50000)

In [ ]:
m = RandomForestRegressor()
%time m.fit(x_trn, y_trn)
print_score(m)    #=> [0.2952, 0.2656, 0.67486495050828088, 0.73733977357618774]

In [ ]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, n_jobs=-1)
%time m.fit(x_trn, y_trn)
print_score(m)    #=> [0.2857, 0.2547, 0.6956, 0.75860000000000005]

In [ ]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
%time m.fit(x_trn, y_trn)
print_score(m)    #=> [0.2856, 0.2536, 0.69569999999999999, 0.76060000000000005]

In [ ]:
m = RandomForestRegressor(n_estimators=50, min_samples_leaf=5, max_features=0.5, n_jobs=-1)
%time m.fit(x_trn, y_trn)
print_score(m)    #=> [0.2825, 0.2498, 0.70220000000000005, 0.76770000000000005]

In [ ]:
reset_rf_samples()
# set_rf_samples(250000)

m = RandomForestRegressor(n_estimators=50, min_samples_leaf=5, max_features=0.5, n_jobs=-1)
%time m.fit(x_trn, y_trn)
print_score(m)    #=> [0.2769, 0.2488, 0.71409999999999996, 0.76949999999999996]

## Variable importances

In [ ]:
features = list(x_trn.columns)

# Get numerical feature importances
importances = list(m.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

## Feature Reduction

In [ ]:
least_important_features = [feature[0] for feature in feature_importances[12:]]

x_trn.drop(least_important_features, axis=1, inplace=True)
x_val.drop(least_important_features, axis=1, inplace=True)

In [ ]:
reset_rf_samples()
set_rf_samples(500000)

m = RandomForestRegressor(n_estimators=50, min_samples_leaf=5, max_features=0.5, n_jobs=-1)
%time m.fit(x_trn, y_trn)
print_score(m)
#=> [0.2769, 0.2488, 0.71409999999999996, 0.76949999999999996]
#=> [0.2732, 0.2487, 0.72150000000000003, 0.76970000000000005]

In [ ]:
features = list(x_trn.columns)

# Get numerical feature importances
importances = list(m.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
reset_rf_samples()
# full dataset - need to run on GPU

m = RandomForestRegressor(n_estimators=100, min_samples_leaf=25, max_features=0.5, n_jobs=8)
%time m.fit(x_trn, y_trn)
print_score(m)
#=> [0.2769, 0.2488, 0.71409999999999996, 0.76949999999999996]
#=> [0.1931, 0.248, 0.8609, 0.7711]  -- overfit?!   (n_est=50, min_leaf=5, 23min)
#=> [0.2484, 0.2466, 0.7699, 0.7737]  (37min)

## Feature Reduction / Data Expansion

In [ ]:
train = pd.read_feather(f'{PATH}rf_rnn/train_w_categories')

In [ ]:
# Extract the names of the least important features
# least_important_features = [feature[0] for feature in feature_importances[15:]]

least_important_features = list(train.columns)[11:-1]
least_important_features.extend(list(['Year', 'Month']))

In [ ]:
train.drop(columns=least_important_features, inplace=True)

In [ ]:
df, y, nas = proc_df(train, 'unit_sales', subset=30000000)

In [ ]:
del train  # need to save memory

In [ ]:
# Using Scikit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
x_trn, x_val, y_trn, y_val = train_test_split(df, y, test_size = 0.20, random_state = 42)

In [ ]:
del df,y

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(x_trn, y_trn)

In [ ]:
print_score(m)   #=> [0.24, 0.5684, 0.946675430780943, 0.7008898192311388]

# slight increase in accuracy (increased data)
# large decrease in duration (feature reduction & split over multiple cores)

### Apply to Test dataset

In [ ]:
test = pd.read_feather(f'{PATH}rf_rnn/test_w_categories')

In [ ]:
test.drop(columns=least_important_features, inplace=True)

In [ ]:
# add in y value to test: 'unit sales'
test['unit_sales'] = 0.0

df_test, _, nas = proc_df(test, 'unit_sales', skip_flds=['id'], na_dict=nas)

In [ ]:
# use rf to predict on test df
log_preds = m.predict(df_test)

In [ ]:
test['unit_sales'] = np.expm1(log_preds) # re-scale predictions and add to df

In [ ]:
SUBM = f'{PATH}rf_rnn/subm/'
os.makedirs(SUBM, exist_ok=True)

test.to_csv(f'{SUBM}rf_v3.csv.gz', columns=['id','unit_sales'], index=False, compression='gzip')

In [ ]:
!kaggle competitions submit -c favorita-grocery-sales-forecasting -f {SUBM}rf_v3.csv.gz -m "random forest version 3"
# v1 => 1.223 -- throwaway
# v2 => 0.581
# v3 => 0.865 (last 3 months) -- throwaway
# v4 => 0.544 (last 3 months)

## Train only on last ~3 months

In [ ]:
train = pd.read_feather(f'{PATH}rf_rnn/train_w_categories')

In [ ]:
# Extract the names of the least important features
# least_important_features = [feature[0] for feature in feature_importances[9:]]

least_important_features = list(train.columns)[11:-1]
least_important_features.extend(list(['Year']))

In [ ]:
train.drop(columns=least_important_features, inplace=True)

In [ ]:
df, y, nas = proc_df(train, 'unit_sales')

In [ ]:
del train  # need to save memory

In [ ]:
m = RandomForestRegressor(n_estimators=100, min_samples_leaf=25, max_features=0.5, n_jobs=8, oob_score=True)
%time m.fit(df, y);

res = [rmsle(m.predict(df), y), round(m.score(df, y),4), m.oob_score_]
print(res)
#=> [0.2453, 0.7755]  do_scale=True
#=> [0.2453, 0.7756, 0.7388747115547389] do_scale=False

## Apply to Test Data

In [ ]:
del x_trn, x_val, y_trn, y_val
# del df, y

In [ ]:
test = pd.read_feather(f'{PATH}rf_rnn/test_w_categories')

In [ ]:
test.drop(columns=least_important_features, inplace=True)

In [ ]:
# add in y value to test: 'unit sales'
test['unit_sales'] = 0.0

In [ ]:
df_test, _, nas, mapper = proc_df(test, 'unit_sales', do_scale=True, skip_flds=['id'], na_dict=nas, mapper=mapper)

In [ ]:
# need to drop columns after scaling because mapper includes all columns
# df_test.drop(columns=least_important_features, inplace=True)

In [ ]:
test['unit_sales'] = m.predict(df_test)  # log1p values
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values

In [ ]:
SUBM = f'{PATH}rf_rnn/subm/'
csv = f'{SUBM}rf_v8.csv.gz'
os.makedirs(SUBM, exist_ok=True)

In [ ]:
test.to_csv(csv, columns=['id','unit_sales'], index=False, compression='gzip')

In [ ]:
!kaggle competitions submit -c favorita-grocery-sales-forecasting -f {csv} -m "RF version 8 - no store_nbr or item_nbr"
# v4 => 0.544 (last 3 months)
# v5 => 0.803
# v6 => 0.801

In [ ]:
test.to_feather(f'{PATH}rf_rnn/test_final_no_items_stores')